In [ ]:
%pip install -U langchain_community langchain_anthropic langchain_experimental matplotlib langgraph
import getpass
import os 

os.environ["ANTHROPIC_API_KEY"] = "ANTHROPIC_API_KEY"
os.environ["TAVILY_API_KEY"] = "TAVILY_API_KEY"
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

tavily_tool = TavilySearchResults(max_results=5)

repl = PythonREPL() 


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code and do math. If you want to see the output of a value,
    you should print it out with print(...). This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return result_str

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re

# Làm sạch HTML
def clean_html(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()
    text = soup.get_text(separator=" ", strip=True)
    return text

# Lấy nội dung từ Facebook
def get_facebook_content(url):
    options = Options()
    options.headless = True
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(5)

    page_source = driver.page_source
    driver.quit()

    soup = BeautifulSoup(page_source, "html.parser")
    post_texts = soup.find_all('div')
    texts = [div.get_text(separator=" ", strip=True) for div in post_texts]
    clean_text = " ".join(texts)
    return clean_text

# Xác định Facebook URL
def is_facebook_url(url):
    return "facebook.com" in url or "m.facebook.com" in url

# Lấy nội dung tuỳ URL
def get_and_clean_content(url):
    try:
        if is_facebook_url(url):
            print(f"Đang lấy dữ liệu từ Facebook: {url}")
            return get_facebook_content(url)
        else:
            print(f"Đang lấy dữ liệu từ website: {url}")
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            return clean_html(response.text)
    except Exception as e:
        return f"Không lấy được dữ liệu từ {url}: {e}"

# Tổng hợp nội dung
def summarize_results(results, score_threshold=0.55):
    filtered_results = [r for r in results if r["score"] > score_threshold]
    combined_content = ""
    for r in filtered_results:
        content = get_and_clean_content(r["url"])
        combined_content += f"\n===== Nội dung từ {r['title']} ({r['url']}) =====\n{content}\n"
    return combined_content

# Gọi Gemini API không giới hạn câu, chỉ yêu cầu ngắn gọn xúc tích
def call_gemini_api(content, query, api_key):
    endpoint = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent?key={api_key}"
    
    headers = {
        "Content-Type": "application/json"
    }

    prompt = f"""
Đây là nội dung tổng hợp từ nhiều nguồn về: "{query}".
Hãy đọc và phân tích toàn bộ nội dung bên dưới và viết một bản tóm tắt ngắn gọn, xúc tích, dễ hiểu, trình bày logic, diễn đạt đầy đủ ý cho đến hết nội dung:

{content}

Hãy trả lời bắt đầu ngay từ nội dung quan trọng nhất.
"""
    data = {
        "contents": [
            {
                "parts": [
                    {"text": prompt}
                ]
            }
        ]
    }

    response = requests.post(endpoint, headers=headers, data=json.dumps(data))
    response_json = response.json()
    return response_json['candidates'][0]['content']['parts'][0]['text']

# Quy trình
query = input("Nhập từ khóa tìm kiếm: ")
results = tavily_tool(query)
summarized_content = summarize_results(results, score_threshold=0.55)

gemini_api_key = "YOUR_API_KEY"

result_from_gemini = call_gemini_api(summarized_content, query, gemini_api_key)

print("\n===== Kết quả tóm tắt từ Gemini =====\n")
print(result_from_gemini)


In [ ]:
query = "4TechGeeks là gì"
tavily_tool(query)